In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import numpy as np
import pandas as pd

In [2]:
Gwangju = pd.read_csv('mini_project_1/Gwangju.csv')
Gwangju

,Unnamed: 0,업종,업소명,주요품목,가격,연락처,주소
0,0,한식_일반,자리봉국밥수완직영점,모듬국밥,9000원,062-952-0555,"광주광역시 광산구 장신로20번길 10, 101호(수완동)"
1,1,한식_일반,도야지에꿈식육식당,삼겹살,14000원,062-681-9292,광주광역시 북구 중외길 30(매곡동)
2,2,이미용업,해나41헤어#,남성컷트,10000원,062-266-5273,광주광역시 북구 참판로 41(풍향동)
3,3,이미용업,효동이용원,컷트,10000원,062-511-2992,광주광역시 북구 군왕로8번길 8(풍향동)
4,4,한식_분식,머꼬요,라면,2500원,0507-1396-3438,광주광역시 북구 우치로 186(용봉동)
...,...,...,...,...,...,...,...
223,223,한식_일반,죽사랑김밥사랑,"찌개(김치,순두부)",5500원,062-227-0988,광주광역시 동구 필문대로 287번길 23-17 (지산동)
224,224,한식_일반,태능회관,"돼지갈비(300g)-숯불, 돌판",15000원,062-234-2884,"광주광역시 동구 천변좌로 722,1 층 (용산동)"
225,225,한식_일반,학동김밥,김치찌개,6000원,062-225-3248,"광주광역시 동구 남문로 683,1층 (학동)"
226,226,한식_일반,대왕김밥(대인동),"찌개(김치,된장,참치,순두부)",5000원,062-529-3363,"광주광역시 동구 구성로204번길 15-6, 1층 (대인동)"


In [3]:
Gwangju.drop('Unnamed: 0',axis = 1, inplace = True)

Gwangju['주소_구,군'] = Gwangju['주소'].apply(lambda x:x.split(' ')[0])

Gwangju['주소_구,군'] = Gwangju['주소'].apply(lambda x:x.replace('\xa0',' '))
Gwangju['keyword'] = Gwangju['주소_구,군']

Gwangju_address_list = Gwangju['keyword'].apply(lambda x:x.split(' '))

new_Gwangju_address_list = []

for i,item in enumerate(Gwangju_address_list):
    #print(ad_li[i][:2])
    address = item[0] + ' ' + item[1]
    new_Gwangju_address_list.append(address)
    
new_Gwangju_address_list

Gwangju['keyword'] = new_Gwangju_address_list

Gwangju['keyword'] = Gwangju['keyword'] + ' ' + Gwangju['업소명']

In [4]:
# 카카오맵 url 가져오기
url = 'https://map.kakao.com/'

# 크롬 드라이버 실행하기
driver = webdriver.Chrome()

# 카카오맵 url로 이동하기
driver.get(url)

score_list = []

for key in Gwangju['keyword'][:301]:
    # 검색창 클릭하기
    search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')

    # keyword의 key를 검색창에 넣기
    search.send_keys(key)

    # 엔터키로 이동하기
    search.send_keys(Keys.RETURN)

    time.sleep(2)

    # 웹 스크립트 받아오기
    html = driver.page_source

    # 웹스크립트에서 원하는 콘텐츠 추출하기 - parsing
    soup = BeautifulSoup(html, 'html.parser')
    
    # keyword별 평점 가져오기
    try:
        # 평점 path 가져오기
        score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
        
        # score_path를 split한 후 3번째 문자 추출하기
        score = soup.select(score_path)[0].text.split('\n')[3]
        
        # 평점을 빈 리스트 L에 추가하기
        score_list.append(score)
        
    except:
        score_list.append('별점 없음')
        
    search.clear()
    
score_list

Gwangju['평점'] = score_list
Gwangju.drop('주소_구,군',axis = 1, inplace = True)
Gwangju.drop('keyword',axis = 1, inplace = True)
Gwangju.to_csv('Gwangjuscore.csv', encoding='utf-8-sig')

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
C:\Users\vvv\AppData\Local\Temp\ipykernel_17416\2279325527.py:14: SyntaxWarning: invalid escape sequence '\.'
  search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
C:\Users\vvv\AppData\Local\Temp\ipykernel_17416\2279325527.py:33: SyntaxWarning: invalid escape sequence '\.'
  score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
